In [1]:
import requests

# URL do arquivo de texto
url = 'https://raw.githubusercontent.com/allanspadini/curso-tensorflow-proxima-palavra/main/dados/texto.txt'

# Fazer o download do conteúdo do arquivo
response = requests.get(url)
texto = response.text

# Dividir o conteúdo do arquivo em uma lista de strings, uma por linha
corpus = texto.splitlines()

In [2]:
from tensorflow.keras.layers import TextVectorization
import tensorflow as tf

In [3]:
max_vocab_size = 2000
max_sequence_len = 50

In [4]:
# Tokenizar o texto
vectorizer = TextVectorization(max_tokens=max_vocab_size, output_sequence_length=max_sequence_len, output_mode='int')
# Adaptar a camada ao corpus
vectorizer.adapt(corpus)
# Convertendo os textos em sequências de tokens
tokenized_corpus = vectorizer(corpus)

In [5]:
# Criar as sequências de n-grams
input_sequences = []
for token_list in tokenized_corpus.numpy():
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [6]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
def prepare_sequences(sequences):
    """
    Prepara as sequências para o modelo, removendo zeros à direita, adicionando padding à esquerda, truncado sequências longas e removendo sequências repetidas.

    Args:
        sequences: Um array de sequências (listas ou arrays NumPy).

    Returns:
        Um array NumPy 2D com as sequências preparadas.
    """

    # Remover zeros à direita de cada sequência
    sequences_without_trailing_zeros = []
    for seq in sequences:
        last_nonzero_index = np.argmax(seq[::-1] != 0)
        if last_nonzero_index == 0 and seq[-1] == 0:
            sequences_without_trailing_zeros.append(np.array([0]))
        else:
            sequences_without_trailing_zeros.append(seq[:-last_nonzero_index or None])

    # Remover sequências repetidas
    unique_sequences = []
    for seq in sequences_without_trailing_zeros:
        if seq.tolist() not in unique_sequences:  # Verifica se a sequência já está na lista
            unique_sequences.append(seq.tolist())  # Adiciona à lista se for única

    # Encontrar o comprimento máximo das sequências sem zeros à direita
    max_sequence_len = max(len(seq) for seq in unique_sequences)

    # Adicionar padding à esquerda para garantir o mesmo comprimento
    padded_sequences = pad_sequences(unique_sequences, maxlen=max_sequence_len, padding='pre', truncating='post')

    return padded_sequences

In [8]:
input_sequences_prepared = prepare_sequences(input_sequences)
print(input_sequences_prepared)

[[  0   0   0 ...   0 449  46]
 [  0   0   0 ... 449  46 437]
 [  0   0   0 ...  46 437  12]
 ...
 [  0   0   0 ... 268   6  47]
 [  0   0   0 ...   6  47 213]
 [  0   0   0 ...  47 213 259]]


In [9]:
# Separar as features (entradas) e o label (saída)
X = input_sequences_prepared[:,:-1]
y = input_sequences_prepared[:,-1]

In [10]:
y = tf.keras.utils.to_categorical(y, num_classes=max_vocab_size)

In [23]:
# Definir o modelo
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=max_vocab_size,
        output_dim=128,
        mask_zero=False
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),  # Camada BiLSTM
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(max_vocab_size, activation='softmax')  # Use total_words em vez de 4 para o número de classes
])

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])



# Treinar o modelo
history = model.fit(X, y, epochs=20, verbose=1, batch_size=64)

Epoch 1/20
56/56 [==============================] - 12s 103ms/step - loss: 7.5560 - accuracy: 0.0099
Epoch 2/20
56/56 [==============================] - 3s 56ms/step - loss: 7.1972 - accuracy: 0.0438
Epoch 3/20
56/56 [==============================] - 2s 42ms/step - loss: 6.4090 - accuracy: 0.0828
Epoch 4/20
56/56 [==============================] - 2s 28ms/step - loss: 5.6172 - accuracy: 0.1252
Epoch 5/20
56/56 [==============================] - 2s 27ms/step - loss: 5.1511 - accuracy: 0.1566
Epoch 6/20
56/56 [==============================] - 2s 37ms/step - loss: 4.8878 - accuracy: 0.1786
Epoch 7/20
56/56 [==============================] - 2s 29ms/step - loss: 4.6832 - accuracy: 0.2029
Epoch 8/20
56/56 [==============================] - 1s 23ms/step - loss: 4.4962 - accuracy: 0.2340
Epoch 9/20
56/56 [==============================] - 1s 25ms/step - loss: 4.3290 - accuracy: 0.2518
Epoch 10/20
56/56 [==============================] - 1s 18ms/step - loss: 4.1384 - accuracy: 0.2719
Epoch 1

In [91]:
def predict_next_words(model, vectorizer, text, max_sequence_len, top_k=3):
    """
    Prediz as próximas palavras mais prováveis em uma sequência de texto.

    Args:
        model: O modelo treinado.
        vectorizer: A camada de vetorização.
        text: O texto de entrada.
        max_sequence_len: O comprimento máximo da sequência usado na vetorização.
        top_k: O número de palavras mais prováveis a serem retornadas.

    Returns:
        As próximas palavras mais prováveis.
    """
    # Vetorizar o texto de entrada
    tokenized_text = vectorizer([text])

    # Remover a dimensão extra adicionada pela vetorização
    tokenized_text = np.squeeze(tokenized_text)

    # Adicionar padding à esquerda
    padded_text = pad_sequences([tokenized_text], maxlen=max_sequence_len, padding='pre')

    # Fazer a previsão
    predicted_probs = model.predict(padded_text, verbose=0)[0]  # Remove a dimensão extra adicionada pela previsão

    # Obter os índices dos top_k tokens com as maiores probabilidades
    top_k_indices = np.argsort(predicted_probs)[-top_k:][::-1]

    # Converter os tokens previstos de volta para palavras
    predicted_words = [vectorizer.get_vocabulary()[index] for index in top_k_indices]

    return predicted_words

In [99]:
text = "Transforme sua vida hoje! Com nosso novo produto, você não apenas alcançará seus objetivos, mas superará suas próprias expectativas. Invista em você    "

In [100]:
predict_next_words(model, vectorizer, text, 50, top_k=10)


['especialistas',
 'inteligente',
 'mais',
 'por',
 'tempo',
 'tudo',
 'sobre',
 'exclusivos',
 'perder',
 'se']

In [98]:
['especialistas',
 'por',
 'mais',
 'inteligente',
 'tudo',
 'tempo',
 'sobre',
 'o',
 'receitas',
 'perder']

['especialistas',
 'por',
 'mais',
 'inteligente',
 'tudo',
 'tempo',
 'sobre',
 'o',
 'receitas',
 'perder']